# Grenade Problem v3.2

Widgets for time of explosion and change in velocity of components

Wesley Terrill {Revised 01-23-2019}

Begin by adding features from libraries

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D #imports 3d plot features

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Tab, Label, Checkbox
from ipywidgets import FloatSlider, IntSlider, Play, Dropdown, HTMLMath 
from ipywidgets import interact, fixed

from IPython.display import display
from time import sleep

Initial velocity of grenade is

$v(t)=(4,0,4)$

Initial position at origin

General trajectories of components 3-vector position

$x(t) = x_0 + v_{0x} t + \frac{a t^2}{2}$ 

$y(t) = y_0 + v_{0y} t + \frac{a t^2}{2}$

$z(t) = z_0 + v_{0z} t - \frac{g t^2}{2}$

Create class with terms to find trajectory and velocity at given time t

In [3]:
#attempt to create class on position
class position:
    def __init__(self, 
                 v_0 = 4.,
                 x_0 = 0.,
                 a_x = 0.):
        
        self.v = v_0
        self.x = x_0
        self.a = a_x
    
    def traj(self,t):
        """
        Horizontal position of grenade
        """
        return self.v * t + self.x + self.a / 2 * t**2
    
    def v_traj(self, t):
        """
        returns velocity at time t
        """
        return self.v + self.a * t
    

Create class for finding 'initial conditions' of trajectory

In [4]:
class find_initial_conditions:
    def __init__(self, 
                 vx=0.,
                 dvx = 0.,
                 x1 = 0.,
                 ax = 0.):
        
        self.v = vx + dvx
        self.x = x1
        self.a = ax

    def initial_v(self, t=0.):
        """
        initial velocity given known values
        """
        return self.v - self.a * t
    
    def x_0(self, tm, v0=0.):
        """
        initial position givem known values
        """
        return self.x - v0 * tm - self.a / 2 * tm**2
    

Create definitions for creating time array's for before and after explosion

In [5]:
def t_before(t_exp):
    """creates time points before explosion"""
    return np.arange(0., t_exp+.1, 1.) # +.1 to get up to t_exp in array

def t_after(t_exp):
    """creates time points after explosion"""
    return np.arange(t_exp, 9.1, 1.)

Creating plotting Definition

In [6]:
def plot_trajectories(dv_x=1.,dv_y=0.,dv_z=3.,t_explode=4.):
    #to plot point locations at specific time t
    tb = t_before(t_explode)
    ta = t_after(t_explode)

    ### Create inital trajectory of grenade ###
### Includes Information about grenade at time of explosion ###
    x_def = position(v_0 = 4.)
    x = x_def.traj(t = tb)
    vx = x_def.v_traj(t = t_explode)
    x_ex = x_def.traj(t = t_explode)
    xa = x_def.traj(t = ta)
    
    y_def = position(v_0 = 0.)
    y = y_def.traj(t = tb)
    vy = y_def.v_traj(t = t_explode)
    y_ex = y_def.traj(t = t_explode)
    ya = y_def.traj(t = ta)
    
    z_def = position(v_0 = 4., a_x = -1.)
    z = z_def.traj(t = tb)
    vz = z_def.v_traj(t = t_explode)
    z_ex = z_def.traj(t = t_explode)
    za = z_def.traj(t = ta)
    
    ### Create trajectories for pieces ###
        ### Piece one ###
    x1_initial = find_initial_conditions(vx=vx, dvx=dv_x, x1=x_ex)
    v_0x1 = x1_initial.initial_v(t = t_explode)
    x_01 = x1_initial.x_0(v0 = v_0x1, tm = t_explode)
    x1_def = position(x_0 = x_01, v_0 = v_0x1)
    x1 = x1_def.traj(t = ta)
    
    y1_initial = find_initial_conditions(vx=vy, dvx=dv_y, x1=y_ex)
    v_0y1 = y1_initial.initial_v(t=t_explode)
    y_01 = y1_initial.x_0(v0 = v_0y1, tm = t_explode)
    y1_def = position(v_0 = v_0y1, x_0 = y_01)
    y1 = y1_def.traj(t = ta)
    
    z1_initial = find_initial_conditions(vx=vz, dvx=dv_z, x1=z_ex, ax=-1.)
    v_0z1 = z1_initial.initial_v(t=t_explode)
    z_01 = z1_initial.x_0(v0 = v_0z1, tm = t_explode)
    z1_def = position(v_0 = v_0z1, x_0 = z_01, a_x = -1.)
    z1 = z1_def.traj(t = ta)
    
    ### Piece two ###
    x2_initial = find_initial_conditions(vx=vx, dvx=-dv_x, x1=x_ex)
    v_0x2 = x2_initial.initial_v(t = t_explode)
    x_02 = x2_initial.x_0(v0 = v_0x2, tm = t_explode)
    x2_def = position(x_0 = x_02, v_0 = v_0x2)
    x2 = x2_def.traj(t = ta)
    
    y2_initial = find_initial_conditions(vx=vy, dvx=-dv_y, x1=y_ex)
    v_0y2 = y2_initial.initial_v(t=t_explode)
    y_02 = y2_initial.x_0(v0 = v_0y2, tm = t_explode)
    y2_def = position(v_0 = v_0y2, x_0 = y_02)
    y2 = y2_def.traj(t = ta)
    
    z2_initial = find_initial_conditions(vx=vz, dvx=-dv_z, x1=z_ex, ax=-1.)
    v_0z2 = z2_initial.initial_v(t=t_explode)
    z_02 = z2_initial.x_0(v0 = v_0z2, tm = t_explode)
    z2_def = position(v_0 = v_0z2, x_0 = z_02, a_x = -1.)
    z2 = z2_def.traj(t = ta)


    mpl.rcParams.update({'font.size':16})
    fig = plt.figure(figsize=(15,15))
    ax = fig.gca(projection='3d')
    
    ax.plot(x, y, z,
            'ro-', #grenade before explosion trajectory
            label=r'initial trajectory');
    
    #ax.plot(xa, ya, za,
     #       'ko-', #CM
      #      label=r'initial trajectory post');


    ax.plot(x1, y1, z1, 'mo-', #grenade piece one trajectory
            label=r'piece one after explosion')  

    ax.plot(x2, y2, z2,'co-', #grenade piece two trajectory
            label=r'piece two after explosion')
    
    ax.plot(.5*(x1+x2),#center of mass trajectory after explosion
            .5*(y1+y2),
            .5*(z1+z2),
            'r:o', label=r'Center of Mass Trajectory');
    
    for i in range(len(ta)):
        ax.plot([x1[i], x2[i]],
                [y1[i], y2[i]],
                [z1[i], z2[i]],
                'k--'
               ) 
    
    ax.set_xlabel('x(t)')
    ax.set_ylabel('y(t)')
    ax.set_zlabel('z(t)')
    ax.set_ylim(-10,10)
    ax.legend(loc='lower right');
    ax.set_aspect('equal');
    
    #for angle in range(0,360):
    #    ax.view_init(30,angle)
    #    plt.draw()
    #    plt.pause(.001)
    


Create Plot with widgets in 3D

In [7]:
traj_widget_x = widgets.FloatSlider(min=-1., max=10.0, step=0.1, value=1.,
                                     description=r'$dv_x$',
                                     readout_format='.2f',
                                     continuous_update=False
                                  )

traj_widget_y = widgets.FloatSlider(min=-1., max=10.0, step=0.1, value=0.,
                                     description=r'$dv_y$',
                                     readout_format='.2f',
                                     continuous_update=False
                                  )

traj_widget_z = widgets.FloatSlider(min=-1., max=10.0, step=0.1, value=3.,
                                     description=r'$dv_z$',
                                     readout_format='.2f',
                                     continuous_update=False
                                  )

time_explode_widget = widgets.FloatSlider(min = 0., max = 7., step = 1., value=4.,
                                    description=r'$t_{explode}$',
                                    readout_format='.2f',
                                    continuous_update=False
                                  )

interact(plot_trajectories,dv_x=traj_widget_x,
         dv_y=traj_widget_y,
         dv_z=traj_widget_z,
         t_explode=time_explode_widget);


interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='$dv_x$', max=10.0, min=-1.0…